In [231]:
!pip install --quiet feedparser langchain_community langgraph duckduckgo-search
!pip install streamlit


In [232]:
# Displaying final output format
from IPython.display import display, Markdown, Latex
import feedparser
from langchain.tools import BaseTool
from urllib.parse import urlencode 

# LangChain Dependencies
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langgraph.graph import END, StateGraph
# For State Graph 
from typing_extensions import TypedDict
import os


In [233]:
# Environment Variables
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ["LANGCHAIN_PROJECT"] = "L3 Research Agent"

In [234]:
# Defining LLM
local_llm = 'llama3.2'
llama3 = ChatOllama(model=local_llm, temperature=0)
llama3_json = ChatOllama(model=local_llm, format='json', temperature=0)

In [235]:
# Web Search Tool

wrapper = DuckDuckGoSearchAPIWrapper(max_results=25)
web_search_tool = DuckDuckGoSearchRun(api_wrapper=wrapper)

# Test Run
resp = web_search_tool.invoke("pei min")
resp

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=4aaf83fc-be7f-4b34-a295-37b8bab5de60,id=4aaf83fc-be7f-4b34-a295-37b8bab5de60


'Pei Min\'s swordsmanship is like his personality, unrestrained and uninhibited. He traveled around the world, doing justice and left countless moving stories. Some people say that Pei Min\'s swordsmanship is like the wind, changing rapidly, just like his unruly heart. He danced with a sword on the streets of Chang\'an, attracting countless people ... Pei Min, on the other hand, was a famous swordsman of the Tang Dynasty, skilled in swordsmanship, and known as the "Sage of Sword". Although they lived in the same era and had different identities, they were both outstanding in their respective fields. Their relationship has become a much-talked-about tale among later generations. Pei Min took off his filial piety and danced with a sword, which amazed the onlookers. When Pei Min\'s sword fell, Wu Daozi painted the picture "for the magnificence of the world". The sword is one of the four famous ancient weapons. It can not only be used to kill the enemy, but also ancient scholars and schola

In [236]:
import json
import requests
from bs4 import BeautifulSoup
from langchain_core.tools import BaseTool

def get_top_hf_papers(n: int):
    url = "https://huggingface.co/papers"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to retrieve papers: {response.status_code}")

    soup = BeautifulSoup(response.text, "html.parser")
    papers = soup.find_all("article")

    paper_info = []
    for paper in papers:
        title = paper.find("h3").text.strip() if paper.find("h3") else "No Title"
        link = paper.find("a")["href"] if paper.find("a") else "#"
        vote_info = paper.find("div", {"class": "flex flex-wrap items-center gap-2.5 pt-1"})
        vote_count = 0
        if vote_info:
            vote_text = vote_info.find("div", {"class": "leading-none"})
            vote_count = int(vote_text.text.strip()) if vote_text and vote_text.text.strip().isdigit() else 0
        thumbnail = paper.find("img")["src"] if paper.find("img") else ""
        author_list = paper.find("ul", {"class": "flex items-center flex-row-reverse text-sm"})

        authors = []
        if author_list:
            for author in author_list.find_all("li"):
                if author.has_attr("title"):
                    authors.append(author["title"])

        paper_info.append({
            "title": title,
            "link": link,
            "votes": vote_count,
            "thumbnail": thumbnail,
            "authors": ", ".join(authors) if authors else "Unknown",
        })

    paper_info.sort(key=lambda x: x["votes"], reverse=True)
    top_papers = paper_info[:n]

    for i, paper in enumerate(top_papers):
        paper_url = f"https://huggingface.co{paper['link']}"
        paper_response = requests.get(paper_url)
        if paper_response.status_code != 200:
            continue
        paper_soup = BeautifulSoup(paper_response.text, "html.parser")
        date_div = paper_soup.find("div", {"class": "mb-6 flex flex-wrap gap-2 text-sm text-gray-500 max-sm:flex-col sm:items-center sm:text-base md:mb-8"})
        published_date = ""
        if date_div and date_div.find("div"):
            published_date = date_div.find("div").text.split("Published on ")[-1].strip()
        abstract_div = paper_soup.find("div", {"class": "pb-8 pr-4 md:pr-16"})
        abstract = abstract_div.find("p").text.strip() if abstract_div and abstract_div.find("p") else "No abstract available"

        top_papers[i]["published_date"] = published_date
        top_papers[i]["abstract"] = abstract

    return json.dumps(top_papers, indent=2)


class HuggingFacePapersAPIWrapper(BaseTool):
    name: str = "huggingface"
    description: str = "Fetch top-voted Hugging Face papers"

    def _run(self, n: int):
        raw = get_top_hf_papers(n)
        return {"source": "huggingface", "results": json.loads(raw)}


hf_tool = HuggingFacePapersAPIWrapper()


In [237]:
def arxiv_search(query: str, max_results: int = 5):
    """
    Query arXiv’s API and return the top N most recent papers.
    """
    base_url = "http://export.arxiv.org/api/query"
    params = {
        "search_query": query,
        "start":        0,
        "max_results":  max_results,
        "sortBy":       "submittedDate",
        "sortOrder":    "descending",
    }
    url = f"{base_url}?{urlencode(params)}"
    feed = feedparser.parse(url)

    results = []
    for entry in feed.entries:
        results.append({
            "title": entry.title,
            "summary": entry.summary,
            "authors": [a.name for a in entry.authors],
            "published": entry.published,
            "pdf_url": next((l.href for l in entry.links if l.type=="application/pdf"), None)
        })
    return results

class ArxivAPIWrapper(BaseTool):
    name: str = "research-search"
    description: str = "Use this to fetch the latest arXiv papers on a topic."

    def _run(self, query: str):
        papers = arxiv_search(query, max_results=5)
        return "\n\n".join(
            f"{p['title']} ({p['published']})\n"
            f"{p['summary'][:300]}…\nPDF: {p['pdf_url']}"
            for p in papers
        )

    async def _arun(self, query: str):
        return self._run(query)

# instantiate
research_tool = ArxivAPIWrapper()

tools = [
    web_search_tool,
    research_tool,     
    hf_tool
]


In [238]:
# Generation Prompt

generate_prompt = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|> 
    
    You are an AI assistant for Research Question Tasks, that synthesizes web search results. 
    Strictly use the following pieces of web search context to answer the question. If you don't know the answer, just say that you don't know. 
    keep the answer concise, but provide all of the details you can in the form of a research report. 
    Only make direct references to material if provided in the context.
    
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    
    Question: {question} 
    Web Search Context: {context} 
    Answer: 
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "context"],
)

# Chain
generate_chain = generate_prompt | llama3 | StrOutputParser()

# Test Run
# question = "How are you?"
# context = ""
# generation = generate_chain.invoke({"context": context, "question": question})
# print(generation)


In [239]:
# Router

router_prompt = PromptTemplate(
    template="""
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>
    You are an expert at routing a user question into one of four categories:
    1. RESEARCH     – user needs the latest peer-reviewed papers (scholarly queries).
    2. NEWS         – user is asking about current events or recent news.
    3. GENERAL      – user should get a direct answer from the LLM without external lookups.
    4. HUGGINGFACE  – user wants trending models or papers on Hugging Face.

    Return JSON with a single key "choice" whose value is exactly one of RESEARCH, NEWS, GENERAL, or HUGGINGFACE.

    Question to route: {question}
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    
    """,
    input_variables=["question"],
)

# Chain
question_router = router_prompt | llama3_json | JsonOutputParser()

# Test Run
question = "top ai model"
print(question_router.invoke({"question": question}))

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=4aaf83fc-be7f-4b34-a295-37b8bab5de60,id=4aaf83fc-be7f-4b34-a295-37b8bab5de60; trace=ae3fa61a-f24f-48fe-8ae3-3dc297a30a00,id=ae3fa61a-f24f-48fe-8ae3-3dc297a30a00; trace=ae3fa61a-f24f-48fe-8ae3-3dc297a30a00,id=c0aad389-0ad3-4873-aef2-e1ad25ea7930; trace=ae3fa61a-f24f-48fe-8ae3-3dc297a30a00,id=c0aad389-0ad3-4873-aef2-e1ad25ea7930; trace=ae3fa61a-f24f-48fe-8ae3-3dc297a30a00,id=671d36ad-663b-405c-9e7a-ee65d99f3a07


{'choice': 'GENERAL'}


In [240]:
# Query Transformation

query_prompt = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|> 
    
    You are an expert at crafting web search queries for research questions.
    More often than not, a user will ask a basic question that they wish to learn more about, however it might not be in the best format. 
    Reword their query to be the most effective web search string possible.
    Return the JSON with a single key 'query' with no premable or explanation. 
    
    Question to transform: {question} 
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>
    
    """,
    input_variables=["question"],
)

# Chain
query_chain = query_prompt | llama3_json | JsonOutputParser()

# Test Run
question = "What's happened recently with chatgpt?"
print(query_chain.invoke({"question": question}))



Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=ae3fa61a-f24f-48fe-8ae3-3dc297a30a00,id=671d36ad-663b-405c-9e7a-ee65d99f3a07; trace=ae3fa61a-f24f-48fe-8ae3-3dc297a30a00,id=5bce9145-61cc-40a4-9f5d-bae8f8585c04; trace=ae3fa61a-f24f-48fe-8ae3-3dc297a30a00,id=5bce9145-61cc-40a4-9f5d-bae8f8585c04; trace=ae3fa61a-f24f-48fe-8ae3-3dc297a30a00,id=ae3fa61a-f24f-48fe-8ae3-3dc297a30a00; trace=473809a4-1c47-4926-9929-065bba110d52,id=473809a4-1c47-4926-9929-065bba110d52; trace=473809a4-1c47-4926-9929-065bba110d52,id=ad76a04d-ec8d-462a-91ed-5e25ed2ce06f; trace=473809a4-1c47-4926-9929-065bba110d52,id=ad76a04d-ec8d-462a-91ed-5e25ed2ce06f; trace=473809a4-1c47-4926-9929-065bba110d52,id=81f823f7-143d-4a56-a9ea-9ee011f0c7b7


{'query': 'recent developments in ChatGPT'}


In [241]:
# Graph State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        search_query: revised question for web search
        context: web_search result
        n: optional number of top HF papers to fetch
    """
    question : str
    generation : str
    search_query : str
    context : str
    n : int

# Node - Generate

def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        dict: New key added to state, generation, that contains LLM generation
    """
    print("Step: Generating Final Response")
    question = state["question"]
    # Safely grab context (empty if none)
    context = state.get("context", "")

    # Answer Generation
    generation = generate_chain.invoke({"context": context, "question": question})
    return {"generation": generation}

# Node – Hugging Face
hf_tool = HuggingFacePapersAPIWrapper()

def huggingface_node(state):
    question = state["question"]
    import re
    match = re.search(r"top\s+(\d+)", question.lower())
    n = int(match.group(1)) if match else 5
    hf_result = hf_tool.invoke({"n": n})
    return {"context": hf_result}

# Node - Query Transformation

def transform_query(state):
    """
    Transform user question to web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended search query
    """
    
    print("Step: Optimizing Query for Web Search")
    question = state['question']
    gen_query = query_chain.invoke({"question": question})
    search_query = gen_query["query"]
    return {"search_query": search_query}


# Node - Web Search

def web_search(state):
    """
    Web search based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to context
    """

    search_query = state['search_query']
    print(f'Step: Searching the Web for: "{search_query}"')
    
    # Web search tool call
    search_result = web_search_tool.invoke(search_query)
    return {"context": search_result}

def research_node(state):
    """
    Call the ArXiv tool and stash results in state['context'].
    """
    print("Step: Running Research Tool")
    q = state.get("query") or state.get("question")
    papers = research_tool.invoke(q)
    # research_tool.invoke returns a single string; if it were a list you'd join it
    return {"context": papers}



# Conditional Edge, Routing

def route_question(state):
    """
    Route question to one of the tags: RESEARCH, NEWS, or GENERAL.
    """
    print("Step: Routing Query")
    q = state.get("query") or state.get("question")
    choice = question_router.invoke({"question": q})["choice"]

    # Just return the tag
    if choice == "RESEARCH":
        print("Tag: RESEARCH")
        return "RESEARCH"
    elif choice == "HUGGINGFACE":
        print("Tag: HUGGINGFACE")
        return "huggingface"
    elif choice == "NEWS":
        print("Tag: NEWS")
        return "NEWS"
    else:
        print("Tag: GENERAL")
        return "GENERAL"



In [242]:
# Build the nodes
workflow = StateGraph(GraphState)
workflow.add_node("websearch",       web_search)
workflow.add_node("transform_query", transform_query)
workflow.add_node("research",        research_node)
workflow.add_node("huggingface",     huggingface_node)
workflow.add_node("generate",        generate)

# Build the edges
workflow.set_conditional_entry_point(
    route_question,
    {
        "RESEARCH":     "research",
        "huggingface":  "huggingface", 
        "NEWS":         "transform_query",
        "GENERAL":      "generate",
    },
)

workflow.add_edge("transform_query", "websearch")
workflow.add_edge("websearch",       "generate")

workflow.add_edge("research",        "generate")
workflow.add_edge("huggingface",     "generate") 
workflow.add_edge("generate",        END)

# Compile the workflow
local_agent = workflow.compile()


In [243]:
def run_agent(query):
    output = local_agent.invoke({"question": query})
    print("=======")
    display(Markdown(output["generation"]))

In [244]:
# Test it out!
# run_agent("What's up with lope recently?")
#run_agent("How are you?")
#run_agent("Find me the latest arXiv papers on graph neural networks?")
run_agent("What are the top 5 papers on HF today? Please organize them in a clear format and include any available details, thumbnails, etc. Display the images in markdown if they exist. use function call")

Step: Routing Query


Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=473809a4-1c47-4926-9929-065bba110d52,id=81f823f7-143d-4a56-a9ea-9ee011f0c7b7; trace=473809a4-1c47-4926-9929-065bba110d52,id=453f6f99-ce28-4875-8959-308fbde62b27; trace=473809a4-1c47-4926-9929-065bba110d52,id=453f6f99-ce28-4875-8959-308fbde62b27; trace=473809a4-1c47-4926-9929-065bba110d52,id=473809a4-1c47-4926-9929-065bba110d52; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=e24a3e43-d4bf-41c1-9263-9540a98a68d3; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=87642dc6-0a0c-4e43-a3f8-9b092e3ce937; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=90922678-f667-4fa5-a97c-13d359a460f6; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=58144b04-4490-4937-99df-0bfabbc79973; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id

Tag: HUGGINGFACE


Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=cf5e14a7-2d7f-4675-9436-ff5e53c87831; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=37ed08af-bdeb-4bf7-a450-1f844679061b; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=37ed08af-bdeb-4bf7-a450-1f844679061b; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=58144b04-4490-4937-99df-0bfabbc79973; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=90922678-f667-4fa5-a97c-13d359a460f6; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=87642dc6-0a0c-4e43-a3f8-9b092e3ce937; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=537d064f-dce4-4476-8dc7-933b54f67376; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=dc294d97-9a04-4a71-a0db-d7c21c108db1


Step: Generating Final Response


Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=dc294d97-9a04-4a71-a0db-d7c21c108db1; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=537d064f-dce4-4476-8dc7-933b54f67376; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=857ff5c2-5c74-4cfc-aeb5-b589dd7762f5; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=c3d35d8b-36c3-4135-81b2-4779003af747; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=818fa854-88b4-449c-8bbd-9d37afd32ade; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=818fa854-88b4-449c-8bbd-9d37afd32ade; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=54ca3905-8b25-4d2a-b0bc-77a9efaffa48


**Top 5 Papers on HF (Hugging Face) as of Today**

Based on the provided web search context from Hugging Face, here are the top 5 papers in a clear format:

1. **Revisiting Reinforcement Learning for LLM Reasoning from A Cross-Domain Perspective**
	* Title: Revisiting Reinforcement Learning for LLM Reasoning from A Cross-Domain Perspective
	* Link: <https://huggingface.co/papers/2506.14965>
	* Authors: Shibo Hao, Zhoujun Cheng, fengyao1909, koalazf99, tianyang
	* Published Date: June 17
	* Thumbnail: ![thumbnail](https://cdn-thumbnails.huggingface.co/social-thumbnails/papers/2506.14965.png)
	* Abstract: Guru, a diverse RL reasoning corpus, highlights domain-specific training needs and demonstrates improved performance in complex tasks for RL-enhanced LLMs.
2. **EmoNet-Voice: A Fine-Grained, Expert-Verified Benchmark for Speech Emotion Detection**
	* Title: EmoNet-Voice: A Fine-Grained, Expert-Verified Benchmark for Speech Emotion Detection
	* Link: <https://huggingface.co/papers/2506.09827>
	* Authors: Robert Kaczmarczyk, soeren1611, felfri, tourist800, ChristophSchuhmann
	* Published Date: June 11
	* Thumbnail: ![thumbnail](https://cdn-thumbnails.huggingface.co/social-thumbnails/papers/2506.09827.png)
	* Abstract: EmoNet-Voice, a new resource with large pre-training and benchmark datasets, advances speech emotion recognition by offering fine-grained emotion evaluation with synthetic, privacy-preserving audio.
3. **Show-o2: Improved Native Unified Multimodal Models**
	* Title: Show-o2: Improved Native Unified Multimodal Models
	* Link: <https://huggingface.co/papers/2506.15564>
	* Authors: Mike Zheng Shou, Zhenheng Yang, Jinheng Xie
	* Published Date: June 18
	* Thumbnail: ![thumbnail](https://cdn-thumbnails.huggingface.co/social-thumbnails/papers/2506.15564.png)
	* Abstract: Show-o2 leverages autoregressive modeling and flow matching within a 3D causal variational autoencoder to create unified visual representations for multimodal understanding and generation tasks.
4. **Improved Iterative Refinement for Chart-to-Code Generation via Structured Instruction**
	* Title: Improved Iterative Refinement for Chart-to-Code Generation via Structured Instruction
	* Link: <https://huggingface.co/papers/2506.14837>
	* Authors: Yuyang Wang, weiranhuang, sunlichao137, WaltonFuture, dazhiga
	* Published Date: June 15
	* Thumbnail: ![thumbnail](https://cdn-thumbnails.huggingface.co/social-thumbnails/papers/2506.14837.png)
	* Abstract: ChartIR uses structured instruction and iterative refinement to improve MLLM performance in chart-to-code generation by separating visual understanding and code translation tasks.
5. **SonicVerse: Multi-Task Learning for Music Feature-Informed Captioning**
	* Title: SonicVerse: Multi-Task Learning for Music Feature-Informed Captioning
	* Link: <https://huggingface.co/papers/2506.15154>
	* Authors: Abhinaba Roy, dorienh, annabeth97c
	* Published Date: June 18
	* Thumbnail: ![thumbnail](https://cdn-thumbnails.huggingface.co/social-thumbnails/papers/2506.15154.png)
	* Abstract: SonicVerse, a multi-task music captioning model, integrates audio feature detection to enhance caption quality and enable detailed descriptions of music pieces.

Note: The thumbnail images are displayed using Markdown syntax (`![thumbnail](https://cdn-thumbnails.huggingface.co/social-thumbnails/papers/2506.14965.png)`) if available in the web search context.

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=54ca3905-8b25-4d2a-b0bc-77a9efaffa48; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=e1bed48c-00ed-4eba-ba94-285b2ecd3bd5; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=e1bed48c-00ed-4eba-ba94-285b2ecd3bd5; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=c3d35d8b-36c3-4135-81b2-4779003af747; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=857ff5c2-5c74-4cfc-aeb5-b589dd7762f5; trace=e24a3e43-d4bf-41c1-9263-9540a98a68d3,id=e24a3e43-d4bf-41c1-9263-9540a98a68d3
